引入包，定义基本属性

In [1]:
import os
import random
import time
import requests
from openai import OpenAI
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

# 从.env文件中导入账号(ACCOUNT)和密码(PASSWORD)
dotenv_path = "../../.env"
load_dotenv(dotenv_path)
ACCOUNT = os.getenv("HNJD_ACCOUNT")
PASSWORD = os.getenv("HNJD_PASSWORD")

form_data = dict()

print("初始化成功.")

初始化成功.


进入登录页

In [3]:
driver = webdriver.Chrome()
driver.maximize_window()
# 等待2秒
time.sleep(.5)
# 刷新网页
driver.refresh()
# 访问指定网页
driver.get("http://authserver.hnjdzy.net/authserver/login?service=http%3A%2F%2F172.16.101.107%2Fsso%2Fjziotlogin")

print("done")

done


登录后台

In [4]:
driver.find_element_by_id("username").send_keys(ACCOUNT)
driver.find_element_by_id("password").send_keys(PASSWORD)
driver.find_element_by_class_name("auth_login_btn").click()

driver.get("http://172.16.101.107/jwglxt/xspjgl/xspj_cxXspjIndex.html?doType=details&gnmkdm=N401605&layout=default&su=202220220474")
print("get in....")

get in....


定义必要方法

In [5]:
def submit():
    """
    点击提交按钮
    :return:
    """
    driver.find_element(By.ID, "btn_xspj_tj").click()
    driver.find_element(By.ID, "btn_ok").click()
    time.sleep(2)


def save():
    """
    点击保存按钮
    :return:
    """
    driver.find_element(By.ID, "btn_xspj_bc").click()
    driver.find_element(By.ID, "btn_ok").click()
    time.sleep(2)

def random_choice(driver):
    """
    有规则的随机选择radio
    :param driver: 
    :return: 
    """
    # 获取所有的form-group
    form_groups = driver.find_elements(By.CSS_SELECTOR, ".form-group")
    # 随机选择从第五个5开始的form_group，除了最后一个
    random_index = random.randint(5, len(form_groups) - 2)
    selected_form_group = form_groups[random_index]
    # 在选定的form_group中选择第二个radio
    second_radio = selected_form_group.find_elements(
        By.CSS_SELECTOR, 'input[type="radio"]'
    )[1]
    second_radio.click()

print("basic function done.")

basic function done.


In [9]:
def select_teacher(driver):
    """
    选择老师
    :param driver:
    :return:
    """

    need_handle = input("请输入您需要手动评价的老师，使用英文逗号分隔。" 
                        "选择完毕后按回车键继续，为空则全部老师满分随机：")
    print("正在选择老师...")
    # 获取css path为#tempGrid .jqgrow的所有老师的行
    rows = driver.find_elements(By.CLASS_NAME, "jqgrow")
    for i in range(len(rows)):
        # 获取行元素，以确保它们是最新的并且与DOM关联
        try:
            row = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "jqgrow"))
            )[i]

            # 检查行内是否存在title为“提交”的td
            if len(row.find_elements(By.CSS_SELECTOR, "td[title='提交']")) != 0:
                continue
            # 检查行内是否存在title为“已评完”的td
            elif len(row.find_elements(By.CSS_SELECTOR, "td[title='已评完']")) != 0:
                continue
            # 点击该row
            row.click()
            # 等待1秒
            time.sleep(1)
            # 将need_handle分割并转换为整数列表
            need_handle_list = [
                int(j.strip()) for j in need_handle.split(",") if j.strip().isdigit()
            ]
            # 检查i + 1是否在这个列表中
            if (i + 1) in need_handle_list:
                fill_form(driver, True)
            else:
                fill_form(driver, False)
        except StaleElementReferenceException:
            break
            

def fill_form(driver, Handle):
    """
    自动填写表单方法
    :param driver: 
    :param Handle: 
    :return: 
    """
    global all_select

    all_select = False

    # 查找class为form-group的form-group列表
    rows = driver.find_elements(By.CSS_SELECTOR, ".tr-xspj")

    # 收集所有的form_groups
    all_form_groups = []
    for row in rows:
        form_groups = row.find_elements(By.CSS_SELECTOR, ".form-group")
        all_form_groups.extend(form_groups)  # 将这个row的form_groups添加到总列表中

    # 检查是否所有的form_groups中都有被选中的radio元素
    if all(
        form_group.find_element(By.CSS_SELECTOR, 'input[type="radio"]').is_selected()
        for form_group in all_form_groups
    ):
        all_select = True

    if not all_select:
        # 遍历form_groups，并点击其中的第一个radio标签
        for form_group in all_form_groups:
            form_group.find_element(By.CSS_SELECTOR, 'input[type="radio"]').click()
            # 等待0.5秒
            time.sleep(0.5)
        if not Handle:
            # 随机选择一个radio
            random_choice(driver)

    comment = ""
    # 查找name为py的textarea标签，并输入评语
    driver.find_element(By.NAME, "py").send_keys(comment)
    # 点击提交按钮
    submit()
        
print("main logic function done.")

main logic function done.


In [10]:
global all_select
select_teacher(driver)

正在选择老师...
